In [2]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import matplotlib.pyplot as plt

#Currencies
peso_dollar = pdr.get_data_fred("DEXMXUS", "1975-01-01", "2019-12-31")
peso_dollar = peso_dollar.squeeze()
HK_dollar = pdr.get_data_fred("DEXHKUS", "1975-01-01", "2019-12-31")
dollar_pound = pdr.get_data_fred("DEXUSUK", "1975-01-01", "2019-12-31")
dollar_pound = dollar_pound.squeeze()

#Create / currency
HK_GBP = HK_dollar
HK_GBP["DEXHKUK"] = HK_dollar["DEXHKUS"] / dollar_pound
HK_GBP = HK_GBP.drop(columns=["DEXHKUS"])
HK_GBP = HK_GBP.squeeze()


#indexes
KS11 = pdr.get_data_yahoo('^KS11', start='1997-07-01', end='2019-12-31')
KS11 = KS11.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"])
KS11 = KS11.squeeze()
BVSP = pdr.get_data_yahoo('^BVSP', start='1993-04-23', end='2019-12-31')
BVSP = BVSP.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"])
BVSP = BVSP.squeeze()
DAX = pdr.get_data_yahoo('^GDAXI', start='1987-12-30', end='2019-12-31')
DAX = DAX.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"])
DAX = DAX.squeeze()
VBTLX = pdr.get_data_yahoo('VBTLX', start='2001-11-12', end='2019-12-31')
VBTLX = VBTLX.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"])
VBTLX = VBTLX.squeeze()

#Factors
ff_factors = pdr.get_data_famafrench("F-F_Research_Data_Factors_daily", start="1920", end="2019-12-31")[0]
ff_factors_mom = pdr.get_data_famafrench("F-F_Momentum_Factor_daily", start="1920", end="2019-12-31")[0]
# ff_factors = ff_factors.join(ff_factors_mom).dropna()
ff_factors = pd.concat([ff_factors,ff_factors_mom], axis=1)
ff_factors = ff_factors.dropna()
ff_factors.columns = [col.strip(' ') for col in ff_factors.columns]
ff_factors_price = (1+(ff_factors/100)).cumprod()

In [113]:
import numpy as np
import pandas as pd

def summary_statistics(prices):

    d_returns = prices.pct_change().dropna()
    w_returns = prices.resample("W-FRI").last().pct_change().dropna()
    m_returns = prices.resample("M").last().pct_change().dropna()
    q_returns = prices.resample("Q").last().pct_change().dropna()
    y_returns = prices.resample("A").last().pct_change().dropna()
    # Convert prices to returns

    # Enclosed function to find mean, std, skew and kurt: gives annualised stats
    def process(returns, n):
        returns_mu = returns.mean()                         # mean returns per time period
        annual_mu =  returns_mu * n                          # annualised mean returns

        returns_err = returns - returns_mu       
        returns_var = (returns_err ** 2).mean()             # average squared return ( 2nd moment)
        annual_var = returns_var * n                        # annual rescaled variance
        annual_std = np.sqrt(annual_var)                    # annual rescaled std deviation

        returns_mom3 = (returns_err ** 3).mean()
        returns_mom4 = (returns_err ** 4).mean()            # Third and Fourth moments


        annual_skew = (returns_mom3 / returns_var ** (3/2)) #/ np.sqrt(n)  # rescaled Skew and Kurtosis
        annual_kurt = (returns_mom4 / returns_var ** (4/2)) #/ n

        returns_stats = pd.Series([annual_mu, annual_std, annual_skew, annual_kurt], index = ["mean", "std", "skew", "kurt"])
        
        return returns_stats
         
    # Feed in daily, weekly, monthly and quarterly data
    d_stats = process(d_returns, n = 252).rename("Daily")
    w_stats = process(w_returns, n = 52 ).rename("Weekly")
    m_stats = process(m_returns, n = 12 ).rename("Monthly")
    q_stats = process(q_returns, n = 4  ).rename("Quarterly")
    y_stats = process(y_returns, n = 1  ).rename("Annual")

    # Concatenate the series into a dataframe
    stats = pd.concat([d_stats, w_stats, m_stats, q_stats, y_stats], axis=1)

    return stats

In [114]:
a = round(summary_statistics(HK_GBP),3)
print(summary_statistics(HK_GBP).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &      Daily &    Weekly &   Monthly &  Quarterly &    Annual \\
\midrule
mean &   0.030483 &  0.031573 &  0.030766 &   0.030202 &  0.027004 \\
std  &   0.106572 &  0.109615 &  0.109426 &   0.117676 &  0.138505 \\
skew &   0.616992 &  0.670850 &  0.417659 &   0.795916 &  0.756487 \\
kurt &  11.902801 &  7.721188 &  4.864608 &   4.914809 &  3.263226 \\
\bottomrule
\end{tabular}



In [115]:
b = round(summary_statistics(peso_dollar),3)
b

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.076,0.076,0.079,0.081,0.083
std,0.145,0.135,0.144,0.157,0.171
skew,5.119,6.175,4.546,2.987,1.803
kurt,148.950,106.744,45.524,15.473,5.739


In [116]:
e = np.round(summary_statistics(KS11),3)
e

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.085,0.082,0.086,0.104,0.128
std,0.272,0.270,0.276,0.323,0.305
skew,-0.025,-0.144,0.846,1.198,0.105
kurt,8.886,8.045,8.442,9.032,3.031


In [73]:
f = np.round(summary_statistics(VBTLX),3)

In [117]:
c = np.round(summary_statistics(DAX),3)
c

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.105,0.103,0.102,0.108,0.112
std,0.220,0.212,0.203,0.226,0.230
skew,-0.082,-0.351,-0.518,-0.631,-0.653
kurt,8.793,6.865,5.062,4.658,2.785


In [119]:
d = np.round(summary_statistics(BVSP),3)
d

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.386,0.378,0.404,0.475,0.607
std,0.355,0.352,0.437,0.828,2.042
skew,0.950,0.632,2.288,4.475,4.481
kurt,17.757,7.500,15.097,27.744,22.036


In [120]:
g = np.round(summary_statistics(ff_factors_price["Mom"]),3)
g

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.067,0.072,0.074,0.075,0.077
std,0.118,0.137,0.152,0.151,0.154
skew,-1.595,-1.390,-0.915,-0.994,-0.721
kurt,29.865,15.511,11.374,9.295,5.379


In [77]:
i = np.round(summary_statistics(ff_factors_price["SMB"]),3)

In [78]:
h = np.round(summary_statistics(ff_factors_price["HML"]),3)

In [85]:
j = np.round(summary_statistics(ff_factors_price["Mkt-RF"]),3)

In [91]:
table_d = pd.concat([a["Daily"],b["Daily"],c["Daily"],d["Daily"],e["Daily"],f["Daily"],g["Daily"],h["Daily"],i["Daily"],j["Daily"]],axis=1)
table_d.columns = ["HK/GBP", "MXN/USD", "DAX", "BSVP", "KS11", "VBTLX", "Mom", "Value", "Size", "Market"]
table_d = np.round(table_d,3)
table_d.name = "Daily"
print(table_d.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  HK/GBP &  MXN/USD &    DAX &    BSVP &   KS11 &  VBTLX &     Mom &   Value &    Size &  Market \\
\midrule
mean &   0.031 &    0.079 &  0.110 &   0.470 &  0.089 &  0.004 &   0.069 &   0.040 &   0.012 &   0.077 \\
std  &   0.107 &    0.145 &  0.220 &   0.355 &  0.272 &  0.037 &   0.118 &   0.093 &   0.093 &   0.169 \\
skew &   0.617 &    5.119 & -0.082 &   0.950 & -0.025 & -0.102 &  -1.595 &   0.732 &  -0.755 &  -0.121 \\
kurt &  11.903 &  148.950 &  8.793 &  17.757 &  8.886 &  4.478 &  29.865 &  18.748 &  25.650 &  19.601 \\
\bottomrule
\end{tabular}



In [95]:
table_w = pd.concat([a["Weekly"],b["Weekly"],c["Weekly"],d["Weekly"],e["Weekly"],f["Weekly"],g["Weekly"],h["Weekly"],i["Weekly"],j["Weekly"]],axis=1)
table_w.columns = ["HK/GBP", "MXN/USD", "DAX", "BSVP", "KS11", "VBTLX", "Mom", "Value", "Size", "Market"]
table_w = np.round(table_w,3)
table_w.name = "Weekly"
print(table_w.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  HK/GBP &  MXN/USD &    DAX &   BSVP &   KS11 &  VBTLX &     Mom &   Value &    Size &  Market \\
\midrule
mean &   0.032 &    0.079 &  0.108 &  0.458 &  0.086 &  0.005 &   0.074 &   0.042 &   0.012 &   0.080 \\
std  &   0.110 &    0.135 &  0.212 &  0.352 &  0.270 &  0.036 &   0.137 &   0.102 &   0.091 &   0.172 \\
skew &   0.671 &    6.175 & -0.351 &  0.632 & -0.144 & -0.514 &  -1.390 &   1.102 &   0.083 &  -0.358 \\
kurt &   7.721 &  106.744 &  6.865 &  7.500 &  8.045 &  4.162 &  15.511 &  16.147 &  13.124 &   9.143 \\
\bottomrule
\end{tabular}



In [96]:
table_m = pd.concat([a["Monthly"],b["Monthly"],c["Monthly"],d["Monthly"],e["Monthly"],f["Monthly"],g["Monthly"],h["Monthly"],i["Monthly"],j["Monthly"]],axis=1)
table_m.columns = ["HK/GBP", "MXN/USD", "DAX", "BSVP", "KS11", "VBTLX", "Mom", "Value", "Size", "Market"]
table_m = np.round(table_m,3)
print(table_m.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  HK/GBP &  MXN/USD &    DAX &    BSVP &   KS11 &  VBTLX &     Mom &   Value &    Size &  Market \\
\midrule
mean &   0.031 &    0.082 &  0.107 &   0.488 &  0.090 &  0.005 &   0.077 &   0.044 &   0.013 &   0.083 \\
std  &   0.109 &    0.144 &  0.203 &   0.437 &  0.276 &  0.034 &   0.152 &   0.115 &   0.107 &   0.185 \\
skew &   0.418 &    4.546 & -0.518 &   2.288 &  0.846 & -0.255 &  -0.915 &   1.292 &   1.117 &   0.193 \\
kurt &   4.865 &   45.524 &  5.062 &  15.097 &  8.442 &  4.416 &  11.374 &  12.344 &  13.602 &  10.933 \\
\bottomrule
\end{tabular}



In [97]:
table_q = pd.concat([a["Quarterly"],b["Quarterly"],c["Quarterly"],d["Quarterly"],e["Quarterly"],f["Quarterly"],g["Quarterly"],h["Quarterly"],i["Quarterly"],j["Quarterly"]],axis=1)
table_q.columns = ["HK/GBP", "MXN/USD", "DAX", "BSVP", "KS11", "VBTLX", "Mom", "Value", "Size", "Market"]
table_q = np.round(table_q,3)
print(table_q.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  HK/GBP &  MXN/USD &    DAX &    BSVP &   KS11 &  VBTLX &    Mom &   Value &    Size &  Market \\
\midrule
mean &   0.031 &    0.083 &  0.113 &   0.566 &  0.108 &  0.005 &  0.077 &   0.046 &   0.014 &   0.088 \\
std  &   0.118 &    0.157 &  0.226 &   0.828 &  0.323 &  0.033 &  0.151 &   0.135 &   0.116 &   0.220 \\
skew &   0.796 &    2.987 & -0.631 &   4.475 &  1.198 & -0.211 & -0.994 &   1.336 &   1.216 &   1.822 \\
kurt &   4.915 &   15.473 &  4.658 &  27.744 &  9.032 &  2.404 &  9.295 &  11.969 &  10.225 &  19.834 \\
\bottomrule
\end{tabular}



In [98]:
table_a = pd.concat([a["Annual"],b["Annual"],c["Annual"],d["Annual"],e["Annual"],f["Annual"],g["Annual"],h["Annual"],i["Annual"],j["Annual"]],axis=1)
table_a.columns = ["HK/GBP", "MXN/USD", "DAX", "BSVP", "KS11", "VBTLX", "Mom", "Value", "Size", "Market"]
table_a = np.round(table_a,3)
print(table_a.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  HK/GBP &  MXN/USD &    DAX &    BSVP &   KS11 &  VBTLX &    Mom &  Value &   Size &  Market \\
\midrule
mean &   0.027 &    0.083 &  0.112 &   0.607 &  0.128 &  0.005 &  0.077 &  0.045 &  0.014 &   0.084 \\
std  &   0.139 &    0.171 &  0.230 &   2.042 &  0.305 &  0.025 &  0.154 &  0.130 &  0.117 &   0.197 \\
skew &   0.756 &    1.803 & -0.653 &   4.481 &  0.105 & -0.050 & -0.721 &  0.353 &  0.226 &  -0.298 \\
kurt &   3.263 &    5.739 &  2.785 &  22.036 &  3.031 &  2.846 &  5.379 &  3.470 &  4.180 &   2.982 \\
\bottomrule
\end{tabular}



In [101]:
KS11_2019 = summary_statistics(KS11["31/12/2009":"31/12/2019"])
KS11_2019 = KS11_2019.drop(columns="Annual")
KS11_2019 = np.round(KS11_2019,4)
KS11_2019

,Daily,Weekly,Monthly,Quarterly
mean,0.0383,0.0367,0.0412,0.0342
std,0.1479,0.1420,0.1299,0.1165
skew,-0.3750,-0.6276,-0.5454,-0.5877
kurt,6.8910,5.0810,4.3409,3.6521


In [34]:
KS11_2009 = summary_statistics(KS11["31/12/1999":"31/12/2009"])
KS11_2009 = KS11_2009.drop(columns="Annual")
KS11_2009 = np.round(KS11_2009,4)
KS11_2009

,Daily,Weekly,Monthly,Quarterly
mean,0.0955,0.1052,0.1002,0.1177
std,0.2957,0.2916,0.2723,0.2998
skew,-0.3692,-0.1743,-0.1789,0.3144
kurt,7.1155,5.9873,3.1254,3.0786


In [9]:
KS11_1999 = summary_statistics(KS11["31/12/1989":"31/12/1999"])
KS11_1999 = KS11_1999.drop(columns="Annual")
KS11_1999 = np.round(KS11_1999,4)
KS11_1999

,Daily,Weekly,Monthly,Quarterly
mean,0.2685,0.2545,0.3398,0.5481
std,0.4768,0.4828,0.5697,0.7275
skew,0.2972,-0.1298,0.6885,0.2167
kurt,3.9634,3.7448,3.2353,2.4770


from scipy.special import gammaln
from scipy.optimize import minimize

def students_dof(prices):
    def std_t_loglik(nu, x):
        # These are fixed for now
        mu = x.mean()
        sigma2 = x.var()
        sigma = np.sqrt(sigma2)

        a = gammaln((nu + 1) / 2)
        b = gammaln(nu / 2)
        c = np.sqrt(np.pi * (nu-2))
        d = ((nu + 1) / 2)
        e = (x - mu) **2
        f = sigma2 * (nu - 2)

        loglik = a - b - np.log(c) - np.log(sigma) - d * np.log(1 + e / f)
        return -(loglik.sum())
    
    d_returns = prices.pct_change().dropna()
    w_returns = prices.resample("W-FRI").last().pct_change().dropna()
    m_returns = prices.resample("M").last().pct_change().dropna()
    q_returns = prices.resample("Q").last().pct_change().dropna()

    starting_val = np.array([3]) 
    opt = minimize(std_t_loglik, starting_val, args=(w_returns), bounds=[(2.01, 100)], options={"disp": True})
    mle = opt.x[0]   
    return mle

In [43]:
def dof(prices):
    
    def calc_kurt(series):
        mu = series.mean()
        err = series - mu
        var = (err**2).mean()
        mom4 = (err**4).mean()
        kurt = mom4 / var ** (4/2)
        return kurt
    
    d_k = calc_kurt(prices.pct_change())
    w_k = calc_kurt(prices.resample("W-FRI").last().pct_change())
    m_k = calc_kurt(prices.resample("M").last().pct_change())
    q_k = calc_kurt(prices.resample("Q").last().pct_change())

    def df(k):
        a = k - 2
        b = k - 4
        df = 3 * (a/b)
        if k > 4:
            df=df
        else:
            df = float("inf")
        return df
    
    df_d = df(d_k)
    df_w = df(w_k)
    df_m = df(m_k)
    df_q = df(q_k)

    freedom = pd.Series([df_d, df_w, df_m, df_q], index = ["Daily", "Weekly", "Monthly", "Quarterly"])
    return freedom




In [44]:
df_DAX = dof(DAX)

In [45]:
df_BVSP = dof(BVSP)

In [46]:
df_HK = dof(HK_GBP)

In [47]:
df_KS11 = dof(KS11)

In [48]:
df_VBTLX = dof(VBTLX)

In [49]:
df_peso = dof(peso_dollar)

In [50]:
df_mom = dof(ff_factors_price["Mom"])

In [51]:
df_hml = dof(ff_factors_price["HML"])

In [52]:
df_smb = dof(ff_factors_price["SMB"])

In [53]:
df_mkt= dof(ff_factors_price["Mkt-RF"])

In [121]:
table = pd.concat([df_HK, df_peso, df_DAX, df_BVSP, df_KS11, df_VBTLX, df_mom, df_hml, df_smb, df_mkt], axis=1)
table = np.round(table,3)
table.columns = ["HK/GBP", "MXN/USD", "DAX", "BSVP", "KS11", "VBTLX", "Momentum", "Value", "Size", "Market"]
table

,HK/GBP,MXN/USD,DAX,BSVP,KS11,VBTLX,Momentum,Value,Size,Market
Daily,3.759,3.041,4.252,3.436,4.228,15.552,3.232,3.407,3.277,3.385
Weekly,4.612,3.058,5.094,4.714,4.483,40.074,3.521,3.494,3.658,4.167
Monthly,9.940,3.144,8.651,3.541,4.351,17.411,3.814,3.719,3.625,3.865
Quarterly,9.559,3.523,12.125,3.253,4.192,inf,4.133,3.753,3.964,3.379
